# Registry workflows

In [1]:
# set lifemonitor root path
lifemonitor_root = "/home/simleo/git/life_monitor"
%cd -q {lifemonitor_root}

In [2]:
import requests

In [4]:
# LifeMonitor URLs
lm_base_url = "https://localhost:8000"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [6]:
# Copy the values of CLIENT_ID and CLIENT_SECRET from the output of "flask registry show seek"
CLIENT_ID = "g2ioTF0AzjFopXbRpSqPc2d2"
CLIENT_SECRET = "PFYk1MUypSVgczX3cfncPmvfNARJnUe9TUJD23JPgaAyxnXL"

In [7]:
# fetch the token
s = requests.session()
s.verify = False
s.headers.update({})
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "read write"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'pD0oUCBBzp8C9MzJ1Iy111658GEPbdp2hxDgB2Gc3U',
 'expires_in': 864000,
 'scope': 'read write',
 'token_type': 'Bearer'}

In [8]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [9]:
# Get workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
registry_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': []}

In [10]:
# Pick a user registry
response = s.get(f"{lm_base_url}/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
registry_users

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'id': 2,
   'identity': {'provider': {'name': 'seek',
     'type': 'seek',
     'uri': 'https://seek:3000',
     'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
    'sub': '2'},
   'username': 'qgkpNNXtqY'}]}

In [12]:
# Pick the first user
assert len(registry_users ) > 0, "Unexpected number of users. We need at least one registered user"
current_user = registry_users['items'][0]
current_user

{'id': 2,
 'identity': {'provider': {'name': 'seek',
   'type': 'seek',
   'uri': 'https://seek:3000',
   'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
  'sub': '2'},
 'username': 'qgkpNNXtqY'}

In [13]:
current_user['identity']['sub']

'2'

In [16]:
wfhub_url = "https://seek:3000"
seek_session = requests.session()
seek_session.verify = False
seek_session.headers.update({
    "Content-type": "application/vnd.api+json",
    "Accept": "application/vnd.api+json",
    "Accept-Charset": "ISO-8859-1"
})
seek_session.auth = requests.auth.HTTPBasicAuth("user1", "workflowhub")
response = seek_session.get(f"{wfhub_url}/people/{current_user['identity']['sub']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
user1 = response.json()["data"]
user1

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': '2',
 'type': 'people',
 'attributes': {'avatar': None,
  'title': 'First User',
  'description': None,
  'first_name': 'First',
  'last_name': 'User',
  'orcid': None,
  'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
  'expertise': None,
  'tools': None,
  'project_positions': None},
 'relationships': {'projects': {'data': [{'id': '2', 'type': 'projects'},
    {'id': '3', 'type': 'projects'}]},
  'institutions': {'data': [{'id': '1', 'type': 'institutions'}]},
  'investigations': {'data': []},
  'studies': {'data': []},
  'assays': {'data': []},
  'data_files': {'data': []},
  'models': {'data': []},
  'sops': {'data': []},
  'publications': {'data': []},
  'presentations': {'data': []},
  'events': {'data': []},
  'documents': {'data': []}},
 'links': {'self': '/people/2'},
 'meta': {'created': '2020-10-06T13:18:25.738Z',
  'modified': '2020-10-06T13:18:25.738Z',
  'api_version': '0.3',
  'uuid': '56f77130-ea04-0138-8cef-0242c0a8ee02',
  'base_url': 'https://seek:

In [19]:
response = seek_session.get(f"{wfhub_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
user_workflows = response.json()["data"]
user_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'id': '9',
  'type': 'workflows',
  'attributes': {'title': 'basefreqsum-invalid'},
  'links': {'self': '/workflows/9'}},
 {'id': '7',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case-invalid'},
  'links': {'self': '/workflows/7'}},
 {'id': '6',
  'type': 'workflows',
  'attributes': {'title': 'basefreqsum'},
  'links': {'self': '/workflows/6'}},
 {'id': '5',
  'type': 'workflows',
  'attributes': {'title': 'sort-and-change-case'},
  'links': {'self': '/workflows/5'}}]

In [20]:
# Get details of the workflow 'sort-and-change-case' (id '5')
response = seek_session.get(f"{wfhub_url}/workflows/5")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()["data"]
workflow

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': '5',
 'type': 'workflows',
 'attributes': {'policy': {'access': 'no_access',
   'permissions': [{'resource': {'id': '3', 'type': 'projects'},
     'access': 'download'}]},
  'title': 'sort-and-change-case',
  'description': 'sort lines and change text to upper case',
  'license': 'Apache-2.0',
  'latest_version': 1,
  'tags': None,
  'discussion_links': None,
  'versions': [{'version': 1,
    'revision_comments': None,
    'url': 'https://seek:3000/workflows/5?version=1'}],
  'version': 1,
  'revision_comments': None,
  'created_at': '2020-10-06T14:37:07.112Z',
  'updated_at': '2020-10-06T14:37:07.112Z',
  'content_blobs': [{'original_filename': 'ro-crate-galaxy-sortchangecase.crate.zip',
    'url': None,
    'md5sum': '008e3381821327156968574cf2567f22',
    'sha1sum': '871152e6fe0908579e89170df75953e420e7b73f',
    'content_type': 'application/zip',
    'link': 'https://seek:3000/workflows/5/content_blobs/5',
    'size': 7087}],
  'other_creators': None,
  'workflow_class': {'t

In [21]:
# Prepare the JSON data required to post the workflow on LifeMonitor
workflow_uuid = workflow['meta']['uuid']
workflow_version = str(workflow["attributes"]["versions"][0]['version']) # pick the first version
workflow_name = workflow["attributes"]["title"]
workflow_roc_link = f'{workflow["attributes"]["content_blobs"][0]["link"]}/download'
post_data = {
    'uuid': workflow_uuid,
    'version': workflow_version,
    'name': workflow_name,
    'roc_link': workflow_roc_link,
    'submitter_id': user1['id']
}
post_data

{'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
 'version': '1',
 'name': 'sort-and-change-case',
 'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
 'submitter_id': '2'}

In [22]:
# trigger the registration of a the workflow 'sort-and-change-case' on LifeMonitor
response = s.post(f"{lm_base_url}/workflows", json=post_data)
assert response.status_code == 201, "Error: status code {} !!!".format(response.status_code)
data = response.json()
data

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'wf_uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04', 'wf_version': '1'}

In [23]:
wf_uuid = data['wf_uuid']
wf_version = data['wf_version']

In [24]:
# Get workflows
response = s.get(f"{lm_base_url}/workflows")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
registry_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'name': 'sort-and-change-case',
   'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
   'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
   'version': '1'}]}

In [25]:
# Get workflow details
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()
workflow

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'name': 'sort-and-change-case',
 'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
 'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
 'version': '1'}

In [26]:
# Get workflow status
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
status = response.json()
status

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'aggregate_test_status': 'all_passing',
 'latest_builds': [{'build_id': '4',
   'instance': {'name': 'test1',
    'service': {'resource': 'job/test/',
     'type': 'jenkins_testing_service',
     'url': 'http://jenkins:8080/',
     'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6',
   'timestamp': '1601997383446'}],
 'workflow': {'name': 'sort-and-change-case',
  'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
  'version': '1'}}

In [27]:
# Get latest version
response = s.get(f"{lm_base_url}/workflows/{wf_uuid}/{wf_version}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_version = response.json()
latest_version

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'name': 'sort-and-change-case',
 'roc_link': 'https://seek:3000/workflows/5/content_blobs/5/download',
 'uuid': '55203d00-ea0f-0138-9780-0242c0a8ee04',
 'version': '1'}

In [28]:
# Get test suites for the workflow
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/{workflow_version}/suites")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suites = response.json()
suites

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'instances': [{'name': 'test1',
     'service': {'resource': 'job/test/',
      'type': 'jenkins_testing_service',
      'url': 'http://jenkins:8080/',
      'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
     'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'}],
   'test_suite_metadata': {'@id': 'test-metadata.json',
    'test': [{'definition': {'path': 'test1/sort-and-change-case-test.yml',
       'test_engine': {'type': 'planemo', 'version': '>=0.70'}},
      'instance': [{'name': 'example_jenkins',
        'service': {'resource': 'job/test/',
         'type': 'jenkins',
         'url': 'http://jenkins:8080/'}}],
      'name': 'test1'}],
    'tmpformat': 'ro/workflow/test-metadata/0.1'},
   'uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6'}]}

In [29]:
# Get the first test suite
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite = response.json()
suite

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'instances': [{'name': 'test1',
   'service': {'resource': 'job/test/',
    'type': 'jenkins_testing_service',
    'url': 'http://jenkins:8080/',
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
   'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'}],
 'test_suite_metadata': {'@id': 'test-metadata.json',
  'test': [{'definition': {'path': 'test1/sort-and-change-case-test.yml',
     'test_engine': {'type': 'planemo', 'version': '>=0.70'}},
    'instance': [{'name': 'example_jenkins',
      'service': {'resource': 'job/test/',
       'type': 'jenkins',
       'url': 'http://jenkins:8080/'}}],
    'name': 'test1'}],
  'tmpformat': 'ro/workflow/test-metadata/0.1'},
 'uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6'}

In [30]:
# Get the suite status
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite_status = response.json()
suite_status

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'latest_builds': [{'build_id': '4',
   'instance': {'name': 'test1',
    'service': {'resource': 'job/test/',
     'type': 'jenkins_testing_service',
     'url': 'http://jenkins:8080/',
     'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6',
   'timestamp': '1601997383446'}],
 'status': 'all_passing',
 'suite_uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6'}

In [31]:
# Get the suite instances
response = s.get(f"{lm_base_url}/suites/{suites['items'][0]['uuid']}/instances")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instances = response.json()
instances

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'name': 'test1',
   'service': {'resource': 'job/test/',
    'type': 'jenkins_testing_service',
    'url': 'http://jenkins:8080/',
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
   'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'}]}

In [32]:
# Get the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instances['items'][0]['uuid']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instance = response.json()
instance

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'name': 'test1',
 'service': {'resource': 'job/test/',
  'type': 'jenkins_testing_service',
  'url': 'http://jenkins:8080/',
  'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
 'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'}

In [33]:
# Get latest builds of the first test suite instance
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/latest-builds")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_builds = response.json()
latest_builds

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'build_id': '4',
   'instance': {'name': 'test1',
    'service': {'resource': 'job/test/',
     'type': 'jenkins_testing_service',
     'url': 'http://jenkins:8080/',
     'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
   'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins2038142812728915165.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
   'status': 'passed',
   'suite_uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6',
   'timestamp': '1601997383446'},
  {'build_id': '3',
   'instance': {'name': 'test1',
    'service': {'resource': 'job/test/',
     'type': 'jenkins_testing_service',
     'url': 'http://jenkins:8080/',
     'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
    'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99

In [34]:
# Get test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
build1

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'build_id': '1',
 'instance': {'name': 'test1',
  'service': {'resource': 'job/test/',
   'type': 'jenkins_testing_service',
   'url': 'http://jenkins:8080/',
   'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
  'uuid': '4c0b3da3-8e75-492e-971a-c210aab8d99c'},
 'last_logs': 'Started by user Admin Jenkins\nRunning as SYSTEM\nBuilding in workspace /var/jenkins_home/workspace/test\n[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh\n+ ls -larth\ntotal 8.0K\ndrwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..\ndrwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .\nFinished: SUCCESS\n',
 'status': 'passed',
 'suite_uuid': '29c74a52-6011-47e6-89c1-1f367e045ad6',
 'timestamp': '1601997377980'}

In [36]:
# Get logs for test build '1'
response = s.get(f"{lm_base_url}/instances/{instance['uuid']}/builds/1/logs")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build1 = response.json()
print(build1)

Started by user Admin Jenkins
Running as SYSTEM
Building in workspace /var/jenkins_home/workspace/test
[test] $ /bin/sh -xe /tmp/jenkins5791607255757933039.sh
+ ls -larth
total 8.0K
drwxr-sr-x 3 jenkins jenkins 4.0K Oct  6 15:16 ..
drwxr-sr-x 2 jenkins jenkins 4.0K Oct  6 15:16 .
Finished: SUCCESS



/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
